In [53]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected

def classification_binary(csvpath,startColumnName,
                          endColumnName,isdefects,importance_result,
                         columncombation_j = None):

    filePath = "AlltheMatrix/"

    filePathNowusing = filePath + csvpath

    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')
    matrixlen = int(len(changed_matrix))
    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == startColumnName):
            startindex = i
        elif (column == endColumnName):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix[endColumnName].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(data.shape)

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]
    print(type(data))
    #split the train set and the test set
    X_train = data[:round(0.7*matrixlen)]
    X_test = data[round(0.7*matrixlen):]
    Y_train = target[:round(0.7*matrixlen)]
    Y_test = target[round(0.7*matrixlen):]
    print(X_train,Y_train.shape)
    #train a binary classifier
    Y_train_bugs = (Y_train > isdefects)
    Y_test_bugs = (Y_test > isdefects)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    if(columncombation_j!=None):
        print("####################################")
        X_train = get_selected_rows(columncombation_j,X_train)
        X_test = get_selected_rows(columncombation_j,X_test)

    forest_clf = RandomForestClassifier(n_estimators = 200)
    forest_clf.fit(X_train,Y_train_bugs)

    cross_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'accuracy')
    recall_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'recall')
    Y_forest_predict = forest_clf.predict(X_train)

    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')
    print(cross_result)
    #    print(X_train_selected)
    Y_test_predict = forest_clf.predict(X_test)

    findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
    print(f'fingdingrate = {findingRate}')
    
    # get the importance of each column
    importances=forest_clf.feature_importances_
    print('the importance of the each column：\n',importances)
    indices = np.argsort(importances)[::-1]
    print('the descend order for importance of the column：\n',indices)
    most_import = indices[:10]
    print(f'most importtant columns:{most_import}')
    print(X_train[:,most_import])
    

    columnlist = np.arange(0,endindex-startindex-1,1)
    for i in columnlist:
        if i in most_import:
            importance_result[i] +=1
    return cross_result,recall_result
#     import csv
#     # have to change the output file name
#     with open('PredictedAlltheMatrix/importance_result.csv','a+') as f:
#         csv_write = csv.writer(f)
#         data_row = [most_import,forst_precision,forst_recall,findingRate,np.mean(cross_result)]
#         csv_write.writerow(data_row)

In [48]:
importance_result = {}
columnlist = np.arange(0,36,1)
for i in columnlist:
    importance_result[i] = 0
for i in range(100):
    classification_binary('all_data.csv','numberOfVersionsUntil','bugs',0,importance_result)
importance_result

AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[1 0 6 ... 2 4 3]
 [1 0 7 ... 12 35 14]
 [0 0 5 ... 1 2 1]
 ...
 [1 0 4 ... 4 52 26]
 [0 0 3 ... 1 25 9]
 [0 0 3 ... 3 30 10]] (1048,)
Forest_accurancy is 1.0
Forest_recall is 0.8543046357615894
[0.87203791 0.85238095 0.86602871 0.87559809 0.86602871]
fingdingrate = (0.1896551724137931, 11)
the importance of the each column：
 [0.00628684 0.00322115 0.01051211 0.03384314 0.02508729 0.02545836
 0.02900137 0.04585056 0.02133336 0.02579261 0.01744718 0.02648103
 0.03081585 0.02184883 0.05657809 0.06339108 0.0620106  0.02403343
 0.05997802 0.02585771 0.01210143 0.02098223 0.03077786 0.02516098
 0.0115082  0.02212103 0.00869432 0.04123734 0.0193674  0.0311269
 0.0166983  0.01950834 0.00193489 0.03271872 0.05309927 0.03813419]
the descend order for importance of the column：
 [15 16 18 14 34  7 27 35  3 33 29 12 22  6 11 19  9  5 23  4 17 25 13  8
 21 31 28 10 30 20 24  2 26  0  1 32]
most importtant 

Forest_accurancy is 1.0
Forest_recall is 0.8866666666666667
[0.86190476 0.88095238 0.84285714 0.85645933 0.85645933]
fingdingrate = (0.23728813559322035, 14)
the importance of the each column：
 [0.00614632 0.00381211 0.01476658 0.03595605 0.03103566 0.02889334
 0.02143484 0.0286068  0.0337823  0.04430346 0.016687   0.03477069
 0.02660581 0.03312563 0.04321924 0.05040789 0.06631023 0.0495838
 0.04286032 0.02122303 0.01943286 0.01640175 0.02026647 0.02675568
 0.00462782 0.01910284 0.00613582 0.04708436 0.02725865 0.01546513
 0.02676326 0.01083428 0.01099646 0.03931457 0.04599606 0.03003291]
the descend order for importance of the column：
 [16 15 17 27 34  9 14 18 33  3 11  8 13  4 35  5  7 28 30 23 12  6 19 22
 20 25 10 21 29  2 32 31  0 26 24  1]
most importtant columns:[16 15 17 27 34  9 14 18 33  3]
[[0.0177997 0.00750328 0.16705699999999998 ... 0.0373325 6 22]
 [0.0266855 0.0279366 0.272522 ... 0.0472278 11 102]
 [0.0135406 0.00543243 0.09869630000000001 ... 0.0368315 2 80]
 ...
 [0.

Forest_accurancy is 1.0
Forest_recall is 0.8918918918918919
[0.87142857 0.85238095 0.87619048 0.86124402 0.84688995]
fingdingrate = (0.18032786885245902, 11)
the importance of the each column：
 [0.00914939 0.00378091 0.02026392 0.03055497 0.02478725 0.02029231
 0.03146085 0.0284505  0.01741898 0.02845006 0.02166581 0.01680767
 0.03799423 0.02639326 0.04194949 0.0571922  0.04499912 0.06289294
 0.04763751 0.02356409 0.02094393 0.01467169 0.03313602 0.03666085
 0.00536863 0.01834637 0.01285696 0.0385622  0.01438721 0.0418918
 0.0228566  0.01947469 0.0063222  0.03748134 0.05018202 0.03115205]
the descend order for importance of the column：
 [17 15 34 18 16 14 29 27 12 33 23 22  6 35  3  7  9 13  4 19 30 10 20  5
  2 31 25  8 11 21 28 26  0 32 24  1]
most importtant columns:[17 15 34 18 16 14 29 27 12 33]
[[0.18625799999999998 0.0177238 4 ... 22 127.429 2]
 [0.24961599999999998 0.11682999999999999 74 ... 108 79.8571 9]
 [0.22438899999999998 0.029387200000000002 80 ... 131 335.286 18]
 ...
 

Forest_accurancy is 1.0
Forest_recall is 0.9078014184397163
[0.8436019  0.86666667 0.85645933 0.8708134  0.88516746]
fingdingrate = (0.17647058823529413, 12)
the importance of the each column：
 [0.01214177 0.00062517 0.01340012 0.01886547 0.03008272 0.02315461
 0.0335007  0.05314651 0.03345218 0.04940718 0.02432197 0.03006553
 0.03882685 0.03451993 0.02311544 0.0362989  0.06277723 0.0343795
 0.06126066 0.02740721 0.02832948 0.0142519  0.02997529 0.01328516
 0.0055831  0.03109009 0.01683116 0.03873944 0.01504437 0.03209784
 0.01423804 0.01733177 0.00918457 0.02610009 0.03535583 0.03181223]
the descend order for importance of the column：
 [16 18  7  9 12 27 15 34 13 17  6  8 29 35 25  4 11 22 20 19 33 10  5 14
  3 31 26 28 21 30  2 23  0 32 24  1]
most importtant columns:[16 18  7  9 12 27 15 34 13 17]
[[0.02671 0.0390316 37 ... 74 15.5722 0.28927800000000004]
 [0.0143904 0.037145599999999994 1 ... 2 34.2857 0.106458]
 [0.0444525 0.06195459999999999 122 ... 17 56.7046 0.496521]
 ...
 [0.

Forest_accurancy is 1.0
Forest_recall is 0.859375
[0.87619048 0.88571429 0.87619048 0.8708134  0.86602871]
fingdingrate = (0.1111111111111111, 9)
the importance of the each column：
 [0.00556919 0.00759442 0.02238764 0.02783724 0.04404373 0.03095289
 0.02682089 0.03395302 0.03489477 0.02106043 0.01812356 0.02266814
 0.03430882 0.03049183 0.03923751 0.06819325 0.03145489 0.03901406
 0.06284835 0.02482237 0.0230254  0.01657661 0.02490114 0.0251587
 0.00277912 0.0244817  0.02366866 0.0288754  0.04000205 0.02900431
 0.03061769 0.01080216 0.00285159 0.01248635 0.05568574 0.02280639]
the descend order for importance of the column：
 [15 18 34  4 28 14 17  8 12  7 16  5 30 13 29 27  3  6 23 22 19 25 26 20
 35 11  2  9 10 21 33 31  1  0 32 24]
most importtant columns:[15 18 34  4 28 14 17  8 12  7]
[[0.00324957 0.03435680000000001 5 ... 3.83333 227.429 14]
 [0.00318815 0.0313813 4 ... 2.71429 335.286 8]
 [0.00015578600000000002 0.0309529 6 ... 1.0 183.857 2]
 ...
 [0.08169839999999999 0.09746439

...

Forest_accurancy is 1.0
Forest_recall is 0.9172413793103448
[0.88095238 0.85238095 0.89047619 0.86124402 0.86124402]
fingdingrate = (0.125, 8)
the importance of the each column：
 [0.01065556 0.0013402  0.01393373 0.03618038 0.03388743 0.03161359
 0.0578161  0.03092605 0.03677996 0.01952389 0.02455477 0.02319477
 0.03477251 0.02908052 0.04166916 0.02692514 0.05553304 0.04701263
 0.06237893 0.02738492 0.02385645 0.0117342  0.01780183 0.02876851
 0.0066975  0.02738601 0.00705928 0.03280615 0.01641772 0.03181022
 0.01688196 0.01782291 0.00508878 0.03590457 0.02930012 0.04550051]
the descend order for importance of the column：
 [18  6 16 17 35 14  8  3 33 12  4 27 29  5  7 34 13 23 25 19 15 10 20 11
  9 31 22 30 28  2 21  0 26 24 32  1]
most importtant columns:[18  6 16 17 35 14  8  3 33 12]
[[0.0 0 0.0 ... 0 2 28.4286]
 [0.037145599999999994 2 0.0143904 ... 2 2 85.2857]
 [0.0515692 30 0.0319751 ... 49 1 144.286]
 ...
 [0.058543700000000004 7 0.0166509 ... 7 3 104.286]
 [0.136011 393 0.0467

Forest_accurancy is 1.0
Forest_recall is 0.8767123287671232
[0.86255924 0.85238095 0.84210526 0.88038278 0.83253589]
fingdingrate = (0.15873015873015872, 10)
the importance of the each column：
 [0.01983694 0.01016003 0.00749468 0.0453899  0.0190955  0.03607951
 0.0360168  0.02718284 0.0382592  0.02630785 0.02143991 0.02072657
 0.03268774 0.02538485 0.04675794 0.04187622 0.03727082 0.05203054
 0.05240103 0.02234879 0.01544124 0.0265363  0.02350776 0.02182764
 0.00931834 0.02049145 0.00695478 0.05135851 0.02407891 0.02672339
 0.0146727  0.01451339 0.00227995 0.03293658 0.04375892 0.04685249]
the descend order for importance of the column：
 [18 17 27 35 14  3 34 15  8 16  5  6 33 12  7 29 21  9 13 28 22 19 23 10
 11 25  0  4 20 30 31  1 24  2 26 32]
most importtant columns:[18 17 27 35 14  3 34 15  8 16]
[[0.035406900000000005 0.21743600000000002 110 ... 0.005304 8.08333
  0.0196272]
 [0.0 0.0 44 ... 0.0 0.0 0.0]
 [0.0543606 0.339617 69 ... 0.06420790000000001 6.923080000000001
  0.033907

Forest_accurancy is 1.0
Forest_recall is 0.8985507246376812
[0.84285714 0.87142857 0.84761905 0.8708134  0.89473684]
fingdingrate = (0.1267605633802817, 9)
the importance of the each column：
 [0.01164541 0.01092632 0.01589959 0.02604646 0.01800548 0.0355902
 0.03338084 0.0326008  0.0274664  0.03423491 0.02281659 0.02501454
 0.03465692 0.03665409 0.04580673 0.05970379 0.05272129 0.06112965
 0.06702844 0.01404801 0.01981472 0.01704574 0.01861365 0.01902455
 0.00377712 0.01722063 0.00704186 0.03250711 0.02448342 0.03387542
 0.01600471 0.01104274 0.00410745 0.03507103 0.04824217 0.02675121]
the descend order for importance of the column：
 [18 17 15 16 34 14 13  5 33 12  9 29  6  7 27  8 35  3 11 28 10 20 23 22
  4 25 21 30  2 19  0 31  1 26 32 24]
most importtant columns:[18 17 15 16 34 14 13  5 33 12]
[[0.035265 0.20523000000000002 0.0068411999999999995 ... 5.0 1
  201.28599999999997]
 [0.0339284 0.0617154 0.000748253 ... 6.0 4 110.0]
 [0.07201210000000001 0.45007700000000006 0.25337 ... 

Forest_accurancy is 1.0
Forest_recall is 0.9019607843137255
[0.86666667 0.85714286 0.85238095 0.8708134  0.8708134 ]
fingdingrate = (0.16071428571428573, 9)
the importance of the each column：
 [0.0075386  0.00198779 0.01515844 0.03086984 0.03647408 0.02136862
 0.04819386 0.02791809 0.03418373 0.02456898 0.01960555 0.02380385
 0.03928165 0.03394975 0.05972319 0.05437325 0.03425175 0.05751329
 0.04209114 0.02015808 0.0234997  0.01672809 0.02569688 0.02291141
 0.01127895 0.02993148 0.0081654  0.02692222 0.0185258  0.03473534
 0.0363497  0.00916376 0.00697379 0.03717603 0.03480081 0.02412709]
the descend order for importance of the column：
 [14 17 15  6 18 12 33  4 30 34 29 16  8 13  3 25  7 27 22  9 35 11 20 23
  5 19 10 28 21  2 24 31 26  0 32  1]
most importtant columns:[14 17 15  6 18 12 33  4 30 34]
[[12.8023 0.343141 0.044066 ... 279 8 83]
 [7.199319999999999 0.205425 0.00596246 ... 17 0 20]
 [0.0 0.0 0.0 ... 0 0 31]
 ...
 [7.10446 0.194702 0.0319829 ... 116 2 26]
 [7.21846 0.213975 

{0: 0,
 1: 0,
 2: 0,
 3: 59,
 4: 19,
 5: 14,
 6: 51,
 7: 43,
 8: 30,
 9: 27,
 10: 11,
 11: 12,
 12: 35,
 13: 14,
 14: 64,
 15: 88,
 16: 94,
 17: 85,
 18: 95,
 19: 4,
 20: 1,
 21: 1,
 22: 13,
 23: 2,
 24: 0,
 25: 6,
 26: 0,
 27: 63,
 28: 3,
 29: 23,
 30: 2,
 31: 0,
 32: 0,
 33: 42,
 34: 64,
 35: 35}

In [50]:
sorted(importance_result.items(),key = lambda item:item[1])[::-1]

[(18, 95),
 (16, 94),
 (15, 88),
 (17, 85),
 (34, 64),
 (14, 64),
 (27, 63),
 (3, 59),
 (6, 51),
 (7, 43),
 (33, 42),
 (35, 35),
 (12, 35),
 (8, 30),
 (9, 27),
 (29, 23),
 (4, 19),
 (13, 14),
 (5, 14),
 (22, 13),
 (11, 12),
 (10, 11),
 (25, 6),
 (19, 4),
 (28, 3),
 (30, 2),
 (23, 2),
 (21, 1),
 (20, 1),
 (32, 0),
 (31, 0),
 (26, 0),
 (24, 0),
 (2, 0),
 (1, 0),
 (0, 0)]

In [59]:
topTenImportantColumn = sorted(importance_result.items(),key = lambda item:item[1])[-7:]
topTenImportantColumn

[(2, 80), (17, 85), (15, 88), (16, 94), (18, 95), (4, 99), (3, 139)]

In [58]:
topTenImportantColumn

[(24, 0),
 (26, 0),
 (31, 0),
 (32, 0),
 (20, 1),
 (21, 1),
 (23, 2),
 (30, 2),
 (28, 3),
 (19, 4),
 (25, 6),
 (10, 11),
 (11, 12),
 (22, 13),
 (5, 14),
 (13, 14),
 (29, 23),
 (9, 27),
 (8, 30),
 (12, 35),
 (35, 35),
 (33, 42),
 (7, 43),
 (6, 51),
 (27, 63),
 (14, 64),
 (34, 64),
 (0, 80),
 (1, 80),
 (2, 80),
 (17, 85),
 (15, 88),
 (16, 94),
 (18, 95),
 (4, 99),
 (3, 139)]

In [60]:
tup = ()
for i,one in enumerate(topTenImportantColumn):
    tup += (one[0],)
print(tup)
a = []
b = []
for i in range(20):
    a +=[ classification_binary('all_data.csv','numberOfVersionsUntil',
                          'bugs',0,importance_result,tup)[0]]
    b += [ classification_binary('all_data.csv','numberOfVersionsUntil',
                          'bugs',0,importance_result,tup)[1]]

(2, 17, 15, 16, 18, 4, 3)
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[0 0 5 ... 11 17 12]
 [0 0 5 ... 3 49 31]
 [1 0 7 ... 5 199 108]
 ...
 [1 0 7 ... 3 11 5]
 [0 0 2 ... 1 8 5]
 [0 0 8 ... 6 51 26]] (1048,)
####################################
Forest_accurancy is 1.0
Forest_recall is 0.9937106918238994
[0.86666667 0.84285714 0.85714286 0.82857143 0.85576923]
fingdingrate = (0.12, 6)
the importance of the each column：
 [0.06890651 0.16372085 0.18192154 0.17267171 0.16083511 0.12100363
 0.13094065]
the descend order for importance of the column：
 [2 3 1 4 6 5 0]
most importtant columns:[2 3 1 4 6 5 0]
[[0.00352932 0.01696 0.182272 ... 24 9 5]
 [0.00703886 0.0163241 0.16933399999999998 ... 107 32 5]
 [0.243971 0.07558510000000002 0.81994 ... 1457 263 7]
 ...
 [0.00786325 0.023407900000000002 0.292636 ... 90 35 7]
 [0.0 0.0 0.0 ... 4 4 2]
 [0.0642518 0.0637418 0.581607 ... 496 153 8]]
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(149

Forest_accurancy is 1.0
Forest_recall is 0.9794520547945206
[0.86729858 0.85714286 0.84688995 0.87559809 0.85645933]
fingdingrate = (0.19047619047619047, 12)
the importance of the each column：
 [0.0633164  0.14873743 0.20046123 0.17211757 0.15695801 0.13079067
 0.12761871]
the descend order for importance of the column：
 [2 3 4 1 5 6 0]
most importtant columns:[2 3 4 1 5 6 0]
[[0.0137629 0.019024 0.0373496 ... 70 94 3]
 [0.153394 0.0570972 0.0818436 ... 254 1806 8]
 [0.00737162 0.0253281 0.0128305 ... 10 25 4]
 ...
 [0.0111182 0.013737799999999998 0.0530614 ... 12 16 1]
 [0.35426199999999997 0.05855880000000001 0.115377 ... 589 2229 5]
 [0.00344743 0.027985000000000003 0.08715139999999999 ... 11 15 3]]
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[0 1 1 ... 2 12 9]
 [3 0 9 ... 8 45 21]
 [1 0 7 ... 3 11 5]
 ...
 [0 0 2 ... 8 39 32]
 [0 0 2 ... 3 17 13]
 [0 0 1 ... 2 7 7]] (1048,)
####################################
Forest_accurancy is 1.

Forest_accurancy is 1.0
Forest_recall is 0.9866666666666667
[0.86666667 0.86190476 0.85714286 0.85645933 0.86124402]
fingdingrate = (0.15254237288135594, 9)
the importance of the each column：
 [0.06442104 0.15686609 0.20049337 0.17444315 0.14619997 0.12260809
 0.13496829]
the descend order for importance of the column：
 [2 3 1 4 6 5 0]
most importtant columns:[2 3 1 4 6 5 0]
[[0.06572 0.043403699999999996 0.500895 ... 314 69 4]
 [0.177304 0.0603233 0.329377 ... 126 23 1]
 [0.0545185 0.0243526 0.308169 ... 549 109 7]
 ...
 [0.0038316 0.0163241 0.16933399999999998 ... 20 7 4]
 [0.010498100000000002 0.017198599999999998 0.167716 ... 78 42 3]
 [0.0 0.0 0.0 ... 0 0 1]]
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[2 0 4 ... 12 22 14]
 [1 1 9 ... 18 47 28]
 [4 0 4 ... 4 135 57]
 ...
 [0 0 2 ... 2 15 9]
 [1 0 8 ... 6 28 15]
 [1 0 3 ... 9 26 21]] (1048,)
####################################
Forest_accurancy is 1.0
Forest_recall is 0.986206896551

Forest_accurancy is 1.0
Forest_recall is 0.9779411764705882
[0.86255924 0.88095238 0.85645933 0.87559809 0.86124402]
fingdingrate = (0.1780821917808219, 13)
the importance of the each column：
 [0.05747145 0.16937597 0.18621447 0.16517828 0.15047976 0.12358718
 0.14769288]
the descend order for importance of the column：
 [2 1 3 4 6 5 0]
most importtant columns:[2 1 3 4 6 5 0]
[[0.0990434 0.568623 0.06654639999999999 ... 793 295 5]
 [0.0345545 0.330129 0.0333089 ... 383 165 5]
 [0.019337599999999996 0.115222 0.0141637 ... 82 55 2]
 ...
 [0.027786500000000002 0.138856 0.018583199999999998 ... 163 105 2]
 [0.0026364 0.16933399999999998 0.0163241 ... 38 10 7]
 [0.0899472 0.30101999999999995 0.0345272 ... 391 176 3]]
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[3 0 2 ... 2 58 36]
 [2 1 3 ... 1 25 6]
 [0 0 3 ... 6 25 10]
 ...
 [0 0 1 ... 4 30 17]
 [0 0 2 ... 1 5 1]
 [0 0 4 ... 2 5 2]] (1048,)
####################################
Forest_accuran

Forest_accurancy is 1.0
Forest_recall is 0.9795918367346939
[0.86255924 0.86190476 0.85645933 0.85645933 0.85645933]
fingdingrate = (0.14516129032258066, 9)
the importance of the each column：
 [0.05485242 0.16973937 0.18556205 0.17657146 0.15175433 0.11887081
 0.14264956]
the descend order for importance of the column：
 [2 3 1 4 6 5 0]
most importtant columns:[2 3 1 4 6 5 0]
[[0.08305069999999999 0.0475214 0.553435 ... 337 117 7]
 [0.00553249 0.0176486 0.20034100000000002 ... 26 10 4]
 [0.147013 0.029402099999999997 0.299304 ... 501 104 4]
 ...
 [0.0370385 0.03557009999999999 0.33115900000000004 ... 120 29 3]
 [0.00514939 0.0158218 0.103073 ... 22 12 1]
 [0.018827700000000003 0.0211768 0.136686 ... 95 66 3]]
AlltheMatrix/all_data.csv
lens: 49
start:7,end:44
(1497,)
(1497, 36)
<class 'numpy.ndarray'>
[[0 1 1 ... 1 4 2]
 [0 0 5 ... 6 56 37]
 [0 0 4 ... 10 72 35]
 ...
 [1 0 4 ... 11 52 41]
 [0 0 4 ... 6 13 8]
 [1 1 9 ... 18 47 28]] (1048,)
####################################
Forest_accur

In [55]:
sum(sum(a)/20)/7,sum(sum(b)/20)/7

(0.8576284784684012, 0.18466724527841044)

In [56]:
a

[array([0.85238095, 0.87142857, 0.86190476, 0.87142857, 0.85576923]),
 array([0.89047619, 0.85714286, 0.84761905, 0.8708134 , 0.88995215]),
 array([0.87203791, 0.85238095, 0.86602871, 0.8277512 , 0.85167464]),
 array([0.85781991, 0.88095238, 0.85645933, 0.84688995, 0.85167464]),
 array([0.83333333, 0.86190476, 0.87619048, 0.8708134 , 0.86602871]),
 array([0.82938389, 0.84761905, 0.86602871, 0.83732057, 0.85645933]),
 array([0.85714286, 0.84761905, 0.86666667, 0.85167464, 0.86124402]),
 array([0.85714286, 0.83809524, 0.87142857, 0.8708134 , 0.88038278]),
 array([0.86190476, 0.85714286, 0.87142857, 0.8277512 , 0.83732057]),
 array([0.85714286, 0.86666667, 0.83333333, 0.85167464, 0.85645933]),
 array([0.84834123, 0.86666667, 0.8277512 , 0.83732057, 0.81818182]),
 array([0.85238095, 0.87619048, 0.86666667, 0.84761905, 0.84615385]),
 array([0.85714286, 0.84761905, 0.86190476, 0.87142857, 0.875     ]),
 array([0.88571429, 0.87142857, 0.85714286, 0.85645933, 0.84210526]),
 array([0.84834123, 